In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
def process_catagorical(df):
    single_catagorical=['make','fuel_type','type_of_vehicle']
    multi_catagorical=['category']

    def to_cat(x,value_list):
        c = value_list.index(x)
        return c


    for h in single_catagorical:
        print(h)
        df[h]=df[h].fillna('none')
        value_list=list(set(df[h].values))
        
        if "none" in value_list:
            value_list.remove("none")
            value_list.sort()
            value_list=["none"]+value_list
        df[h] = df[h].apply(to_cat,args=(value_list,))

    def to_multi_cat(x,value_list):
        c=""
        for i,value in enumerate(value_list):
            if value in x:
                c+="-"+str(i)+"-"
        return c


    for h in multi_catagorical:
        print(h)
        data=list(df[h].fillna('none').unique())
        #print(data)
        catagory=set([])
        for l in data:
            cat_l=l.strip('-').split(",")
            for cat in cat_l:
                if len(cat)!=0:
                    catagory.add(cat.strip(" "))
        value_list=list(catagory)
        value_list.sort()
        print(value_list)
        df[h]=df[h].apply(to_multi_cat,args=(value_list,))
    
        for i in range(len(value_list)):
            df[value_list[i]]=0
            #df=post_process(df)
            df.loc[df['category'].str.contains("-"+str(i)+"-"), value_list[i]]=1
            #df.head(20)

    return df

In [ ]:
def post_process(df):
    df['trans'] = 1
    df[df.transmission == 'manual']['trans'] = 0
    # types = ['luxury sedan', 'sports car', 'suv']
    # df['ordinal_type'] = 0
    # df[df.type_of_vehicle.isin(types)]['ordinal_type'] = 1
    df=process_catagorical(df)
    #df = df.drop(columns = ['features', 'category', 'type_of_vehicle', 'description', 'model', 'make', 'title', 'transmission', 'fuel_type', 'Unnamed: 0'])
    df = df.drop(columns = ['features', 'description', 'model', 'title', 'transmission', 'Unnamed: 0','category'])
    return df

In [ ]:
df = pd.read_csv('./train_num_cleaned.csv')
df=post_process(df)
df.head(20)

In [ ]:
from sklearn.model_selection import train_test_split
#X = df[['power', 'depreciation', 'road_tax', 'dereg_value', 'mileage', 'category_11']]
X = df[[i for i in df.columns if i not in ['price','depreciation','dereg_value']]]
y = df['dereg_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=500,max_features = 5).fit(X_train,y_train)
sorted_idx = model.feature_importances_.argsort()
plt.barh(X_train.columns[sorted_idx], model.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")